# Import des modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy.stats import pearsonr

# Extrait SIRH

- Calcul des statistiques descriptives
- Objectif : faire ressortir des différences clés entre les employés

In [ ]:
bc = pd.read_csv('seatle.csv')


# Extrait Evaluation Performance

- Calcul des statistiques descriptives
- Objectif : faire ressortir des différences clés entre les employés

In [ ]:
bc = pd.read_csv('seatle.csv')


# Extrait Sondage

- Calcul des statistiques descriptives
- Objectif : faire ressortir des différences clés entre les employés

In [ ]:
bc = pd.read_csv('seatle.csv')


# Création du fichier central

- Identifier les colonnes permettant de réaliser des jointures entre les 3 fichiers.

In [ ]:
###########
# PANDAS
##########

##########
# SQL
##########